In [2]:
pip install -r ./requirements.txt

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/523.1 kB ? eta -:--:--
   --- ------------------------------------ 51.2/523.1 kB 1.3 MB/s eta 0:00:01
   ----------- ---------------------------- 153.6/523.1 kB 1.8 MB/s eta 0:00:01
   ------------------------ --------------- 317.4/523.1 kB 2.2 MB/s eta 0:00:01
   -------------------------------- ------- 419.8/523.1 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------  522.2/523.1 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 523.1/523.1 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/164.4 kB ? eta -:--:--
   -------------- ------------------------- 61.4/164.4 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 164.4/164.4 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.5 kB ? eta -:--:--
   ---------------------------------------- 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/147.8 kB ? eta -:--:--


In [3]:
import autogen
from autogen import AssistantAgent, UserProxyAgent

Patching name='__init__', member=<function LLMLingua.__init__ at 0x00000245FE77E840>, patched=<function function.__call__ at 0x00000245FE77DC60>
Patching name='compress_text', member=<function LLMLingua.compress_text at 0x00000245FE77E8E0>, patched=<function function.__call__ at 0x00000245FE77EA20>


In [4]:
open_ai_key =''

In [5]:
config_list = [{'model':'gpt-4o', 'api_key':open_ai_key}]

gpt_config = {
    "cache_speed" :42,
    "temperature" : 0,
    "config_list" : config_list,
    "timeout" : 120
}



In [6]:
task = '''
    **Task**: As an architect, you are required to design a solution for the following business requirements:
    - Data storage for massive amounts of IOT data
    - Real time data analytics and machine learning pipeline 
    - Scalability
    - Cost Optimization
    - Region pairs in EUrope, for disaster recovery
    - Tools for monitoring and observability
    - Timeline : 6 mmonths

    Break down the problem using a chain of tought approach. Ensure that your solution is following 
    the best practices.
    '''

In [15]:
cloud_prompt = '''
**Role**: You are expert cloud architect. You need to develop architecture proposals using either
cloud specific PAAS systems or cloud agnosistic ones. A final architecture should cponsider all 3 main cloud providers: Azure,AWS,GCP
and provide a data architecture for each. At the end biefly state the advantages of cloud over on premises and summarize solution for 
each cloud provider using a table for clarity.
'''

cloud_prompt += task

In [9]:
oss_promt ='''
**Role**: You are an expert on premisses, open source software architect. You need to develop architecture proposals
with out considering cloud providers. Only use open source framewors that are popular and have active contributors.
At the end briefly state the advantages of using open source adoption using a table for clarity
'''

oss_promt += task

In [11]:
lead_prompt = '''
**Role**: You are a lead architect tasked with managing a conversation between the cloud and open source architects.
Each Architect will perform a task and respond with their results. You will critically review their those and also ask
for, or point to, the disadvantages of their solutions. You will review each result and choose the best solution in accordance with the business
requirements and architecture best practices. You will use any number of summary tables to communicate your decision.
'''

lead_prompt +=task


In [14]:
user_proxy = UserProxyAgent(
    name = "Supervisor",
    system_message = "A Human head of Architecture",
    code_execution_config = {
        "last_n_messages" : 2,
        "work_dir" : "groupchat",
        "use_docker" : False,
    },
    human_input_mode = "NEVER"
)

In [16]:
cloud_agent = AssistantAgent(
    name = "cloud",
    system_message = cloud_prompt,
    llm_config = { "config_list": config_list}
)

In [17]:
oss_agent = AssistantAgent(
    name = "oss",
    system_message = oss_promt,
    llm_config = { "config_list": config_list}
)

In [18]:
lead_agent = AssistantAgent(
    name = "lead",
    system_message = lead_prompt,
    llm_config = { "config_list": config_list}
)

In [20]:
def state_transition(last_speaker, groupchat):
    if last_speaker is user_proxy:
        return cloud_agent
    elif last_speaker is cloud_agent:
        return oss_agent
    elif last_speaker is oss_agent:
        return lead_agent
    elif last_speaker is lead_agent:
        return None

In [21]:
llm_config = { "config_list": config_list}

In [22]:
groupchat = autogen.GroupChat(
    agents = [user_proxy, cloud_agent, oss_agent, lead_agent],
    messages = [],
    max_round = 6,
    speaker_selection_method = state_transition
)

In [23]:
manager = autogen.GroupChatManager(groupchat = groupchat, llm_config = llm_config)

In [24]:
user_proxy.initiate_chat(
    manager, message = "Provide your best architecture based on the AI agent for the business requirements.")

Supervisor (to chat_manager):

Provide your best architecture based on the AI agent for the business requirements.

--------------------------------------------------------------------------------

Next speaker: cloud

cloud (to chat_manager):

To address the business requirements for a robust architecture that supports massive IoT data ingestion, real-time analytics, machine learning, scalability, cost optimization, and ensures disaster recovery in Europe, we need to develop a solution for each major cloud provider (AWS, Azure, GCP). Below is a breakdown of the chain of thought used to develop this architecture.

### Chain of Thought

1. **Understanding Data Storage Needs**: 
   - Given the nature of IoT data, which is often massive and diverse, we require highly scalable storage solutions that can handle structured and unstructured data.
   
2. **Real-time Analytics and Machine Learning**:
   - Data needs to be processed in real-time to generate actionable insights, which implies low

ChatResult(chat_id=None, chat_history=[{'content': 'Provide your best architecture based on the AI agent for the business requirements.', 'role': 'assistant', 'name': 'Supervisor'}, {'content': "To address the business requirements for a robust architecture that supports massive IoT data ingestion, real-time analytics, machine learning, scalability, cost optimization, and ensures disaster recovery in Europe, we need to develop a solution for each major cloud provider (AWS, Azure, GCP). Below is a breakdown of the chain of thought used to develop this architecture.\n\n### Chain of Thought\n\n1. **Understanding Data Storage Needs**: \n   - Given the nature of IoT data, which is often massive and diverse, we require highly scalable storage solutions that can handle structured and unstructured data.\n   \n2. **Real-time Analytics and Machine Learning**:\n   - Data needs to be processed in real-time to generate actionable insights, which implies low-latency and high-throughput processing ca